<img src="https://numba.pydata.org/_static/numba-blue-horizontal-rgb.svg" width="40%" align="right">

# Fast vectorization with Numba

`np.vectorize` is a very convenient function but is unfortunately slow. It is only marginally faster than writing a for loop in Python and looping. 

A common way to get around this is to write a base interpolation function that can handle nD arrays in a compiled language like C or Fortran and then pass that to `apply_ufunc`.

Another option is to use the [numba package](https://numba.pydata.org/) which provides two very convenient decorators to build [numpy universal functions or ufuncs](https://numba.readthedocs.io/en/stable/user/vectorize.html):
1. [`vectorize`](https://numba.readthedocs.io/en/stable/user/vectorize.html#the-vectorize-decorator) for functions that act on scalars, and 
2. [`guvectorize`](https://numba.pydata.org/numba-doc/latest/user/vectorize.html#the-guvectorize-decorator) for functions that operates on subsets of the array along core-dimensions. Any decorated function gets compiled and will loop over the loop dimensions in parallel when necessary. 

For `apply_ufunc` the key concept is that we must provide `vectorize=False` (the default) when using Numba vectorized functions. 
Numba handles the vectorization (or looping) and `apply_ufunc` handles converting Xarray objects to bare arrays and handling metadata.

## Load data

In [ ]:
%xmode minimal

import numpy as np
import xarray as xr

da = xr.DataArray(
    np.arange(12).reshape(3, 4),
    dims=("x", "y"),
    coords={"x": [12, 13, 14]},
    attrs={"foo": "bar"},
)
da

## `vectorize`

Our `squared_error` example from earlier works element-by-element, and is a great example for `vectorize`

In [ ]:
from numba import vectorize, float64


@vectorize([float64(float64, float64)])
def squared_error(x, y):
    return (x - y) ** 2

See the numba documentation to understand `@vectorize([float64(float64, float64)])`

Now use `apply_ufunc` to apply it.

In [ ]:
xr.apply_ufunc(squared_error, da, 1)

## `guvectorize`

`guvectorize` is for functions that work on small subsets of the data. Quoting the Numba documentation
> While `vectorize()` allows you to write ufuncs that work on one element at a time, the `guvectorize()` decorator takes the concept one step further and allows you to write ufuncs that will work on an arbitrary number of elements of input arrays, and take and return arrays of differing dimensions. The typical example is a running median or a convolution filter.

This description should remind you of `apply_ufunc`!

We will use the example function `g` from the [numba docs](https://numba.readthedocs.io/en/stable/user/vectorize.html#the-guvectorize-decorator), which adds a scalar `y` to a 1D vector `x`. The `res` argument here will contain the output (this is a Numba detail).


In [ ]:
from numba import guvectorize, int64


@guvectorize([(int64[:], int64, int64[:])], '(n),()->(n)')
def g(x, y, res):
    for i in range(x.shape[0]):
        res[i] = x[i] + y


a = np.arange(5)
g(a, 2)

Unlike `squared_error` we cannot pass an Xarray object to `g` directly.

In [ ]:
g(da, 1)

Now use `apply_ufunc`.

In [ ]:
xr.apply_ufunc(
    g,
    da,
    1,
    input_core_dims=[["x"], []],
    output_core_dims=[["x"]],
)

Notice the following:
1. The `guvectorize` decorator includes the concept of "core dimensions": `'(n),()->(n)'`. This string means that the `g` takes a 1D vector of size `n`, a scalar, and returns a 1D vector of size `n`. There is one core dimension for the input, and one core dimension for the output. Both core dimensions have the same size.
2. That string translates to `input_core_dims=[["x"], []], output_core_dims=[["x"]]` in `apply_ufunc`.
3. We don't provide `vectorize=True` to `apply_ufunc` since `numba` will handle the vectorization in compiled code automatically.

## With dask


Use the chunked DataArray

In [ ]:
da_dask = da.chunk({"y": 1})
da_dask

```{exercise}
:label: g

Apply `g` to `da_dask`
```
````{solution} g
:class: dropdown

```python
xr.apply_ufunc(
    g,
    da_dask, 
    1, 
    input_core_dims=[["x"], []], 
    output_core_dims=[["x"]],
    dask="parallelized",
)
```
````

## Next

For more, see the numpy.interp end-to-end example in the left sidebar.